In [1]:
import os
import logging
from typing import Any, Set


from automata.config.base import AgentConfigName
from automata.singletons.dependency_factory import dependency_factory, DependencyFactory
from automata.singletons.py_module_loader import py_module_loader
from automata.symbol.graph import SymbolGraph
from automata.symbol_embedding.base import SymbolCodeEmbedding
from automata.symbol_embedding.vector_databases import (
    ChromaSymbolEmbeddingVectorDatabase,
)

    
logger = logging.getLogger(__name__)

In [2]:
py_module_loader.reset()
dependency_factory.reset()
project_name = "llama_index"
py_rel_path = "llama_index"
py_module_loader.initialize(f"../../repo_store/{project_name}", py_rel_path) # !!~replace w/ your path~!!


In [3]:
symbol_graph = SymbolGraph(
    os.path.join(DependencyFactory.DEFAULT_SCIP_FPATH, f"{project_name}.scip")
)

code_embedding_db = ChromaSymbolEmbeddingVectorDatabase(
    project_name,
    persist_directory=DependencyFactory.DEFAULT_CODE_EMBEDDING_FPATH,
    factory=SymbolCodeEmbedding.from_args,
)

In [4]:
dependency_factory.set_overrides(
    **{
        "symbol_graph": symbol_graph,
        "code_embedding_db": code_embedding_db,
    }
)


symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get('symbol_code_embedding_handler')


In [5]:
raw_ranks = symbol_search.symbol_rank.get_ranks()
print(f"Demonstrating SymbolRank (on {project_name}) \n")
for i,rank in enumerate(raw_ranks[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with rank {rank[1]:.3f}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2158/2158 [00:38<00:00, 56.58it/s]

Demonstrating SymbolRank (on llama_index) 

rank 0 = llama_index.schema.Document with rank 0.005
rank 1 = llama_index.schema.BaseNode with rank 0.003
rank 2 = llama_index.indices.query.schema.QueryBundle with rank 0.003
rank 3 = llama_index.indices.service_context.ServiceContext with rank 0.002
rank 4 = llama_index.llms.base.ChatMessage with rank 0.002
rank 5 = llama_index.callbacks.base.CallbackManager with rank 0.002
rank 6 = llama_index.indices.base.BaseIndex with rank 0.002
rank 7 = llama_index.schema.TextNode with rank 0.002
rank 8 = llama_index.schema.NodeWithScore with rank 0.002
rank 9 = llama_index.schema.BaseNode.get_content with rank 0.002


In [6]:
query = "How do I make an index?"

embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')
code_embeddings = symbol_code_embedding_handler.get_ordered_entries()

print(f"Demonstrating Similarity Search (on {project_name}) for query = {query} \n")
code_similarity_results = embedding_similarity_calculator.calculate_query_similarity_dict(code_embeddings, query)
for i,rank in enumerate(list(code_similarity_results.items())[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with distance {rank[1]:.3f}")

Demonstrating Similarity Search (on llama_index) for query = How do I make an index? 

rank 0 = llama_index.vector_stores.redis.RedisVectorStore._create_index with distance 0.794
rank 1 = llama_index.vector_stores.tair.TairVectorStore._create_index with distance 0.789
rank 2 = llama_index.vector_stores.milvus.MilvusVectorStore._create_index with distance 0.783
rank 3 = llama_index.vector_stores.myscale.MyScaleVectorStore._create_index with distance 0.768
rank 4 = llama_index.indices.base.BaseIndex.index_id with distance 0.763
rank 5 = llama_index.storage.index_store.types.BaseIndexStore with distance 0.763
rank 6 = llama_index.indices.tree.all_leaf_retriever.TreeAllLeafRetriever.__init__ with distance 0.761
rank 7 = llama_index.indices.tree.tree_root_retriever.TreeRootRetriever.__init__ with distance 0.761
rank 8 = llama_index.readers.faiss.FaissReader.__init__ with distance 0.757
rank 9 = llama_index.vector_stores.redis.RedisVectorStore.delete_index with distance 0.756


In [7]:
symbol_rank_search_results = symbol_search.symbol_rank_search(query)
print(f"Demonstrating SymbolRank (on {project_name}) for query = {query} \n")
for i,rank in enumerate(symbol_rank_search_results[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with rank {rank[1]:.3f}")

Demonstrating SymbolRank (on llama_index) for query = How do I make an index? 

rank 0 = llama_index.schema.Document with rank 0.005
rank 1 = llama_index.indices.base.BaseIndex with rank 0.004
rank 2 = llama_index.schema.BaseNode with rank 0.003
rank 3 = llama_index.data_structs.struct_type.IndexStructType with rank 0.003
rank 4 = llama_index.indices.base.BaseIndex.index_struct with rank 0.003
rank 5 = llama_index.indices.composability.graph.ComposableGraph with rank 0.003
rank 6 = llama_index.vector_stores.redis.RedisVectorStore._create_index with rank 0.003
rank 7 = llama_index.vector_stores.redis.RedisVectorStore with rank 0.003
rank 8 = llama_index.vector_stores.tair.TairVectorStore with rank 0.003
rank 9 = llama_index.indices.vector_store.base.VectorStoreIndex with rank 0.003


In [8]:
# defaults to 'automata' project
project_name = 'automata'
py_module_loader.reset()
dependency_factory.reset()

py_module_loader.initialize()
symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get('symbol_code_embedding_handler')


In [9]:
raw_ranks = symbol_search.symbol_rank.get_ranks()
print(f"Demonstrating SymbolRank (on {project_name}) \n")
for i,rank in enumerate(raw_ranks[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with rank {rank[1]:.3f}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 661/661 [00:02<00:00, 264.92it/s]

Demonstrating SymbolRank (on automata) 

rank 0 = automata.symbol.base.Symbol with rank 0.006
rank 1 = automata.core.base.database.vector.ChromaVectorDatabase with rank 0.005
rank 2 = automata.agent.providers.OpenAIAutomataAgent with rank 0.005
rank 3 = automata.core.base.database.vector.VectorDatabaseProvider with rank 0.005
rank 4 = automata.symbol.graph.SymbolGraph with rank 0.004
rank 5 = automata.core.base.database.vector.JSONVectorDatabase with rank 0.004
rank 6 = automata.retrievers.py.context.PyContextRetriever with rank 0.004
rank 7 = automata.singletons.dependency_factory.DependencyFactory with rank 0.004
rank 8 = automata.symbol_embedding.vector_databases.ChromaSymbolEmbeddingVectorDatabase with rank 0.004
rank 9 = automata.singletons.github_client.RepositoryClient with rank 0.004


In [10]:
embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')
code_embeddings = symbol_code_embedding_handler.get_ordered_entries()

query = "How do I make an agent?"
print(f"Demonstrating Similarity Search (on {project_name}) for query = {query} \n")
code_similarity_results = embedding_similarity_calculator.calculate_query_similarity_dict(code_embeddings, query)
for i,rank in enumerate(list(code_similarity_results.items())[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with distance {rank[1]:.3f}")

Demonstrating Similarity Search (on automata) for query = How do I make an agent? 

rank 0 = automata.agent.agent.Agent with distance 0.777
rank 1 = automata.agent.agent.AgentInstance with distance 0.774
rank 2 = automata.config.base.AgentConfigBuilder.build with distance 0.766
rank 3 = automata.agent.providers.OpenAIAutomataAgent with distance 0.765
rank 4 = automata.config.openai_agent.OpenAIAutomataAgentConfig.get_llm_provider with distance 0.762
rank 5 = automata.cli.commands.run_agent with distance 0.760
rank 6 = automata.agent.agent.AgentInstance.create with distance 0.755
rank 7 = automata.config.base.AgentConfig.get_llm_provider with distance 0.752
rank 8 = automata.tasks.executor.IAutomataTaskExecution._build_agent with distance 0.751
rank 9 = automata.cli.options.agent_options with distance 0.750


In [11]:
symbol_rank_search_results = symbol_search.symbol_rank_search(query)
print(f"Demonstrating SymbolRank (on {project_name}) for query = {query} \n")
for i,rank in enumerate(symbol_rank_search_results[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with rank {rank[1]:.3f}")

Demonstrating SymbolRank (on automata) for query = How do I make an agent? 

rank 0 = automata.agent.providers.OpenAIAutomataAgent with rank 0.011
rank 1 = automata.agent.agent.Agent with rank 0.008
rank 2 = automata.config.openai_agent.OpenAIAutomataAgentConfigBuilder with rank 0.008
rank 3 = automata.agent.agent.AgentInstance with rank 0.008
rank 4 = automata.config.openai_agent.OpenAIAutomataAgentConfig with rank 0.007
rank 5 = automata.config.base.AgentConfig with rank 0.007
rank 6 = automata.cli.scripts.run_agent.main with rank 0.006
rank 7 = automata.config.base.AgentConfigBuilder with rank 0.006
rank 8 = automata.tools.factory.AgentToolFactory with rank 0.006
rank 9 = automata.config.openai_agent.OpenAIAutomataAgentConfigBuilder.create_from_args with rank 0.006
